In [5]:
import math
import struct
import inspect
from dataclasses import dataclass
from typing import Any, Optional, Tuple

import numpy as np
import torch
import torch.nn.functional as F
from torch import nn


@dataclass
class ModelArgs:
    # default hyperparameters for the Llama 7B model
    dim: int = 4
    n_layers: int = 4
    n_heads: int = 1
    n_kv_heads: Optional[int] = None
    vocab_size: int = 1024
    hidden_dim: Optional[int] = None
    multiple_of: int = 8
    norm_eps: float = 1e-5
    max_seq_len: int = 1024
    dropout: float = 0.0
    max_batch_size: int = 1



def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0):
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    t = torch.arange(end, device=freqs.device)  # type: ignore
    freqs = torch.outer(t, freqs).float()  # type: ignore
    freqs_cos = torch.cos(freqs)  # real part
    freqs_sin = torch.sin(freqs)  # imaginary part
    return freqs_cos, freqs_sin


def reshape_for_broadcast(freqs_cis: torch.Tensor, x: torch.Tensor):
    ndim = x.ndim
    assert 0 <= 1 < ndim
    assert freqs_cis.shape == (x.shape[1], x.shape[-1])
    shape = [d if i == 1 or i == ndim - 1 else 1 for i, d in enumerate(x.shape)]
    return freqs_cis.view(shape)


def apply_rotary_emb(
    xq: torch.Tensor,
    xk: torch.Tensor,
    freqs_cos: torch.Tensor,
    freqs_sin: torch.Tensor
) -> Tuple[torch.Tensor, torch.Tensor]:

    # reshape xq and xk to match the complex representation
    xq_r, xq_i = xq.float().reshape(xq.shape[:-1] + (-1, 2)).unbind(-1)
    xk_r, xk_i = xk.float().reshape(xk.shape[:-1] + (-1, 2)).unbind(-1)

    # reshape freqs_cos and freqs_sin for broadcasting
    freqs_cos = reshape_for_broadcast(freqs_cos, xq_r)
    freqs_sin = reshape_for_broadcast(freqs_sin, xq_r)

    # apply rotation using real numbers
    xq_out_r = xq_r * freqs_cos - xq_i * freqs_sin
    xq_out_i = xq_r * freqs_sin + xq_i * freqs_cos
    xk_out_r = xk_r * freqs_cos - xk_i * freqs_sin
    xk_out_i = xk_r * freqs_sin + xk_i * freqs_cos

    # flatten last two dimensions
    xq_out = torch.stack([xq_out_r, xq_out_i], dim=-1).flatten(3)
    xk_out = torch.stack([xk_out_r, xk_out_i], dim=-1).flatten(3)

    return xq_out.type_as(xq), xk_out.type_as(xk)



def repeat_kv(x: torch.Tensor, n_rep: int) -> torch.Tensor:
    """torch.repeat_interleave(x, dim=2, repeats=n_rep)"""
    bs, slen, n_kv_heads, head_dim = x.shape
    if n_rep == 1:
        return x
    return (
        x[:, :, :, None, :]
        .expand(bs, slen, n_kv_heads, n_rep, head_dim)
        .reshape(bs, slen, n_kv_heads * n_rep, head_dim)
    )



class Attention(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.n_kv_heads = args.n_heads if args.n_kv_heads is None else args.n_kv_heads
        assert args.n_heads % self.n_kv_heads == 0
        model_parallel_size = 1
        self.n_local_heads = args.n_heads // model_parallel_size
        self.n_local_kv_heads = self.n_kv_heads // model_parallel_size
        self.n_rep = self.n_local_heads // self.n_local_kv_heads
        self.head_dim = args.dim // args.n_heads
        self.wq = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        self.wk = nn.Linear(args.dim, self.n_kv_heads * self.head_dim, bias=False)
        self.wv = nn.Linear(args.dim, self.n_kv_heads * self.head_dim, bias=False)
        self.wo = nn.Linear(args.n_heads * self.head_dim, args.dim, bias=False)
        self.attn_dropout = nn.Dropout(args.dropout)
        self.resid_dropout = nn.Dropout(args.dropout)
        self.dropout = args.dropout
        
        mask = torch.full((1, 1, args.max_seq_len, args.max_seq_len), float("-inf"))
        mask = torch.triu(mask, diagonal=1)
        self.register_buffer("mask", mask)
        
        self.cache_k = torch.zeros(
            (
                args.max_batch_size,
                args.max_seq_len,
                self.n_kv_heads,
                self.head_dim,
            )
        )
        self.cache_v = torch.zeros(
            (
                args.max_batch_size,
                args.max_seq_len,
                self.n_kv_heads,
                self.head_dim,
            )
        )

    def forward(
        self,
        x: torch.Tensor,
        freqs_cos: torch.Tensor,
        freqs_sin: torch.Tensor,
        start_pos: torch.Tensor,
        use_cache: bool = True
    ):
        bsz, seqlen, _ = x.shape

        # QKV
        xq, xk, xv = self.wq(x), self.wk(x), self.wv(x)
        xq = xq.view(bsz, seqlen, self.n_local_heads, self.head_dim)
        xk = xk.view(bsz, seqlen, self.n_local_kv_heads, self.head_dim)
        xv = xv.view(bsz, seqlen, self.n_local_kv_heads, self.head_dim)

        # RoPE relative positional embeddings
        xq, xk = apply_rotary_emb(xq, xk, freqs_cos, freqs_sin)
        
        if use_cache:
            self.cache_k[:bsz, start_pos : start_pos + seqlen] = xk
            self.cache_v[:bsz, start_pos : start_pos + seqlen] = xv

            ks = self.cache_k[:bsz, : start_pos + seqlen]
            vs = self.cache_v[:bsz, : start_pos + seqlen]
        else:
            ks = xk
            vs = xv
        
        # grouped multiquery attention: expand out keys and values
        xk = repeat_kv(ks, self.n_rep)  # (bs, seqlen, n_local_heads, head_dim)
        xv = repeat_kv(vs, self.n_rep)  # (bs, seqlen, n_local_heads, head_dim)

        # make heads into a batch dimension
        xq = xq.transpose(1, 2)  # (bs, n_local_heads, seqlen, head_dim)
        xk = xk.transpose(1, 2)
        xv = xv.transpose(1, 2)
        
        # https://pytorch.org/docs/stable/generated/torch.nn.functional.scaled_dot_product_attention.html
#         mask = self.mask[0, 0, :seqlen, :seqlen]
#         output = torch.nn.functional.scaled_dot_product_attention(
#             xq, xk, xv, attn_mask=mask, dropout_p=self.dropout if self.training else 0.0
#         )
        
        scores = torch.matmul(xq, xk.transpose(2, 3)) / math.sqrt(self.head_dim)
        scores = scores + self.mask[:, :, :seqlen, :seqlen]   # (bs, n_local_heads, seqlen, cache_len + seqlen)
        scores = F.softmax(scores.float(), dim=-1).type_as(xq)
        scores = self.attn_dropout(scores)
        output = torch.matmul(scores, xv)
        
        # restore time as batch dimension and concat heads
        output = output.transpose(1, 2).contiguous().view(bsz, seqlen, -1)

        # final projection into the residual stream
        output = self.wo(output)
        output = self.resid_dropout(output)
        return output

In [6]:
params = ModelArgs()

In [7]:
freqs_cos, freqs_sin = precompute_freqs_cis(params.dim // params.n_heads, params.max_seq_len)
freqs_cos.shape, freqs_sin.shape

(torch.Size([1024, 2]), torch.Size([1024, 2]))

## Context+Cache

In [8]:
# KV Cache发生的地方就在Attention
model = Attention(params)
torch.manual_seed(42)
x = torch.rand(1, 5, 4) # bs, seq_len, dim
model(x, freqs_cos[:5], freqs_sin[:5], 0, True)

tensor([[[-0.2590,  0.3917, -0.2154, -0.1412],
         [-0.2325,  0.3910, -0.2234, -0.1140],
         [-0.1691,  0.1946, -0.0948, -0.1264],
         [-0.1622,  0.1424, -0.0596, -0.1409],
         [-0.1475,  0.1171, -0.0438, -0.1316]]], grad_fn=<UnsafeViewBackward0>)

In [9]:
# KV Cache已被填充，L=5个
model.cache_k[0, :10, 0, 0]

tensor([ 0.3081, -0.2316, -0.5535, -0.3219,  0.1441,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000], grad_fn=<SelectBackward0>)

## Context+Current+Cache

In [12]:
# 这一轮先生成4个
model = Attention(params)
torch.manual_seed(42)
x = torch.rand(1, 5, 4)
x1 = x[:, :4, :]
x2 = x[:, 4:5, :]
model(x1, freqs_cos[:4], freqs_sin[:4], 0, True)
# 注意看，这个输出和刚刚的前4个是一样的

tensor([[[-0.2590,  0.3917, -0.2154, -0.1412],
         [-0.2325,  0.3910, -0.2234, -0.1140],
         [-0.1691,  0.1946, -0.0948, -0.1264],
         [-0.1622,  0.1424, -0.0596, -0.1409]]], grad_fn=<UnsafeViewBackward0>)

In [13]:
# 看新的KV Cache，和前面的前4个依然是一样的
model.cache_k[0, :10, 0, 0]

tensor([ 0.3081, -0.2316, -0.5535, -0.3219,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000], grad_fn=<SelectBackward0>)

In [14]:
# 然后我们生成第5个Token，生成的就是最开始的第5行
model(x2, freqs_cos[4:5], freqs_sin[4:5], 4, True)

tensor([[[-0.1475,  0.1171, -0.0438, -0.1316]]], grad_fn=<UnsafeViewBackward0>)

In [15]:
# 再来看KV Cache，第5个元素来了！
model.cache_k[0, :10, 0, 0]

tensor([ 0.3081, -0.2316, -0.5535, -0.3219,  0.1441,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000], grad_fn=<SelectBackward0>)

## Context+NoCache

In [16]:
# 现在不使用KV Cache
model = Attention(params)
torch.manual_seed(42)
x = torch.rand(1, 5, 4)
model(x, freqs_cos[:5], freqs_sin[:5], 0, False)
# 结果和使用了Cache是一样的

tensor([[[-0.2590,  0.3917, -0.2154, -0.1412],
         [-0.2325,  0.3910, -0.2234, -0.1140],
         [-0.1691,  0.1946, -0.0948, -0.1264],
         [-0.1622,  0.1424, -0.0596, -0.1409],
         [-0.1475,  0.1171, -0.0438, -0.1316]]], grad_fn=<UnsafeViewBackward0>)

In [17]:
# Cache没有
model.cache_k[0, :10, 0, 0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])